# An Analysis of Covid 19 and Vaccination Progress in Europe: a Data Warehouse approach

The dataset used in this project is downlaoded form https://www.ecdc.europa.eu/en/data/downloadable-datasets.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sqlite3
import csv

In [ ]:
conn = sqlite3.connect('covid_data_analysis.db')
cursor = conn.cursor()

In [ ]:
conn.execute('''CREATE TABLE pandemic
             (L_ID INTEGER,
              country TEXT,
              region TEXT,
              T_ID INTEGER,
              month TEXT,
              quarter TEXT,
              cases INTEGER,
              deaths INTEGER,
              PRIMARY KEY (L_ID, T_ID),
              FOREIGN KEY (T_ID) REFERENCES time(T_ID))''')

In [ ]:
conn.execute('''CREATE TABLE vaccination
             (Vacc_ID INTEGER,
              Vaccination_Rate REAL,
              T_ID INTEGER,
              month TEXT,
              month_ID INTEGER,
              quarter TEXT,
              L_ID INTEGER,
              country TEXT,
              region TEXT,
              FirstDose INTEGER,
              vaccine TEXT,
              population INTEGER,
              PRIMARY KEY (Vacc_ID, T_ID),
              FOREIGN KEY (T_ID) REFERENCES time(T_ID))''')

In [ ]:
conn.execute('''CREATE TABLE ICU
             (L_ID INTEGER,
              country TEXT,
              region TEXT,
              T_ID INTEGER,
              month TEXT,
              quarter TEXT,
              source TEXT,
              ICU_Cases INTEGER,
              PRIMARY KEY (L_ID, T_ID),
              FOREIGN KEY (T_ID) REFERENCES time(T_ID))''')

In [ ]:
conn.execute('''CREATE TABLE time
             (T_ID INTEGER NOT NULL PRIMARY KEY,
              month TEXT,
              month_ID INTEGER,
              quarter_ID INTEGER,
              FOREIGN KEY (month_ID) REFERENCES month(month_ID),
              FOREIGN KEY (quarter_ID) REFERENCES quarter(quarter_ID))''')

In [ ]:
conn.execute('''CREATE TABLE month
             (month_ID INTEGER NOT NULL PRIMARY KEY,
              quarter_ID INTEGER,
              month TEXT,
              FOREIGN KEY (quarter_ID) REFERENCES quarter(quarter_ID))''')

In [ ]:
conn.execute('''CREATE TABLE quarter
             (quarter_ID INTEGER PRIMARY KEY,
             quarter TEXT)''')

In [ ]:
conn.execute('''CREATE TABLE location
             (L_ID INTEGER NOT NULL PRIMARY KEY,
              country_ID INTEGER,
              FOREIGN KEY (country_ID) REFERENCES country(country_ID))''')

In [ ]:
conn.execute('''CREATE TABLE country
             (country_ID INTEGER NOT NULL PRIMARY KEY,
              country TEXT,
              region_ID INTEGER,
              FOREIGN KEY (region_ID) REFERENCES region(region_ID))''')

In [ ]:
conn.execute('''CREATE TABLE region
             (region_ID INTEGER NOT NULL PRIMARY KEY,
              region_code TEXT)''')

In [ ]:
df_pandemic= pd.read_csv('PANDEMIC.csv')

In [ ]:
df_pandemic.head()

,month,cases,deaths,countriesAndTerritories
0,10,3557.0,0.0,Austria
1,10,5494.0,4.0,Austria
2,10,7776.0,4.0,Austria
3,10,8221.0,6.0,Austria
4,10,10007.0,8.0,Austria


In [ ]:
df_pandemic = df_pandemic.rename(columns={'countriesAndTerritories':'country'})

In [ ]:
df_pandemic.head()

,month,cases,deaths,country
0,10,3557.0,0.0,Austria
1,10,5494.0,4.0,Austria
2,10,7776.0,4.0,Austria
3,10,8221.0,6.0,Austria
4,10,10007.0,8.0,Austria


In [ ]:
for i in range(1, 10000):
    conn.execute("INSERT INTO pandemic (month,  cases, deaths, country) VALUES (?, ?, ?, ?)", (i, i*2, i*4, i*8))
    conn.commit()

In [ ]:
df_vaccination = pd.read_csv('VACCINATION.csv')

In [ ]:
df_vaccination.head()

,month_ID,month,FirstDose,region,vaccine,population
0,1,January,0,AT,UNK,8901064
1,2,February,0,AT,SGSK,8901064
2,3,March,0,AT,JANSS,8901064
3,4,April,0,AT,COMBA.4-5,8901064
4,5,May,0,AT,MODBA.1,8901064


In [ ]:
for i in range(1, 10000):
    conn.execute("INSERT INTO vaccination (month_ID,	month,	FirstDose,	region,	vaccine,	population) VALUES (?, ?, ?, ?, ?, ?)", (i, i*2, i*4, i*8, i*16, i*32))
    conn.commit()

In [ ]:
df_icu = pd.read_csv('ICU.csv')

In [ ]:
df_icu.head()

,country,value,source
0,Austria,856.0,Country_Website
1,Austria,823.0,Country_Website
2,Austria,829.0,Country_Website
3,Austria,826.0,Country_Website
4,Austria,712.0,Country_Website


In [ ]:
df_icu = df_icu.rename(columns={'value':'ICU_Cases'})

In [ ]:
df_icu.head()

,country,ICU_Cases,source
0,Austria,856.0,Country_Website
1,Austria,823.0,Country_Website
2,Austria,829.0,Country_Website
3,Austria,826.0,Country_Website
4,Austria,712.0,Country_Website


In [ ]:
for i in range(1, 10000):
    conn.execute("INSERT INTO icu (country,	ICU_Cases ,	source) VALUES (?, ?, ?)", (i, i*2, i*4))
    conn.commit()

In [ ]:
import os

database_file = 'covid_data_analysis.db'

# Get the size of the pandemic table
pandemic_size = os.path.getsize(database_file) - os.path.getsize(database_file ) - os.path.getsize(database_file)

# Get the size of the vaccination table
vaccination_size = os.path.getsize(database_file) - os.path.getsize(database_file) - os.path.getsize(database_file)

# Get the size of the ICU table
icu_size = os.path.getsize(database_file) - os.path.getsize(database_file ) - os.path.getsize(database_file )

print(f"Size of pandemic table: {pandemic_size} bytes")
print(f"Size of vaccination table: {vaccination_size} bytes")
print(f"Size of ICU table: {icu_size} bytes")


Size of pandemic table: -1282048 bytes
Size of vaccination table: -1282048 bytes
Size of ICU table: -1282048 bytes


In [ ]:
create_table_query = """
CREATE TABLE covid_data (
    Date DATE,
    Region TEXT,
    Cases INTEGER,
    Deaths INTEGER,
    ICU_Cases INTEGER,
    Vaccination_Rate REAL,
    Vaccine TEXT,
    Vaccinations INTEGER
)
"""

cursor.execute(create_table_query)

conn.commit()


In [ ]:
query = """
SELECT Region, strftime('%m-%Y', Date) AS Month,
       SUM(Cases) AS Total_Cases,
       SUM(Deaths) AS Total_Deaths,
       SUM(ICU_Cases) AS Total_ICU_Cases,
       AVG(Vaccination_Rate) AS Avg_Vaccination_Rate,
       SUM(CASE WHEN Vaccine = 'Pfizer' THEN Vaccinations ELSE 0 END) AS Pfizer_Vaccinations,
       SUM(CASE WHEN Vaccine = 'Moderna' THEN Vaccinations ELSE 0 END) AS Moderna_Vaccinations,
       SUM(CASE WHEN Vaccine = 'Johnson & Johnson' THEN Vaccinations ELSE 0 END) AS JNJ_Vaccinations
FROM covid_data
GROUP BY Region, Month
"""

cursor.execute(query)

results = cursor.fetchall()

for row in results:
    print(row)


In [ ]:
# Insert data from the pandemic.csv file
with open('pandemic.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # skip the header row
    for row in reader:
        cursor.execute('''
            INSERT INTO covid_data (region, date, cases, deaths, source)
            VALUES (?, ?, ?, ?, ?)
        ''', (row[0], row[1], row[2], row[3], 'pandemic'))

# Insert data from the vaccination.csv file
with open('vaccination.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # skip the header row
    for row in reader:
        cursor.execute('''
            INSERT INTO covid_data (region, date, vaccine, doses_administrated, source)
            VALUES (?, ?, ?, ?, ?)
        ''', (row[0], row[1], row[2], row[3], 'vaccination'))

# Insert data from the icu.csv file
with open('icu.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # skip the header row
    for row in reader:
        cursor.execute('''
            INSERT INTO covid_data (region, date, icu_cases, source)
            VALUES (?, ?, ?, ?)
        ''', (row[0], row[1], row[2], 'icu'))

# Save the changes to the database
conn.commit()


In [ ]:
cursor.execute('''
    ALTER TABLE covid_data
    ADD COLUMN source TEXT
''')

# Save the changes to the database and close the connection
conn.commit()

In [ ]:
# Add the doses_administrated column to the covid_data table if it does not exist
cursor.execute('''
    ALTER TABLE covid_data
    ADD COLUMN doses_administrated INTEGER
''')

# Save the changes to the database and close the connection

NameError: name 'cursor' is not defined

In [ ]:
query = "SELECT * FROM covid_data LIMIT 300"
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

(3557, '10', 0, 'Austria', None, None, None, None, 'pandemic', None)
(5494, '10', 4, 'Austria', None, None, None, None, 'pandemic', None)
(7776, '10', 4, 'Austria', None, None, None, None, 'pandemic', None)
(8221, '10', 6, 'Austria', None, None, None, None, 'pandemic', None)
(10007, '10', 8, 'Austria', None, None, None, None, 'pandemic', None)
(13204, '10', 7, 'Austria', None, None, None, None, 'pandemic', None)
(9964, '10', 8, 'Austria', None, None, None, None, 'pandemic', None)
(6606, '10', 12, 'Austria', None, None, None, None, 'pandemic', None)
(8818, '10', 6, 'Austria', None, None, None, None, 'pandemic', None)
(11751, '10', 10, 'Austria', None, None, None, None, 'pandemic', None)
(13068, '10', 14, 'Austria', None, None, None, None, 'pandemic', None)
(14305, '10', 13, 'Austria', None, None, None, None, 'pandemic', None)
(18498, '10', 11, 'Austria', None, None, None, None, 'pandemic', None)
(13369, '10', 10, 'Austria', None, None, None, None, 'pandemic', None)
(8689, '10', 10, 'Aus

In [ ]:
# QUESTION 5: CASES AND DEATHS FOR EACH REGION AND MONTH
# Create a view to report cases by region and month
conn.execute('''
    CREATE VIEW cases_by_region_months AS
    SELECT region, strftime('%Y-%m', date) AS month,
           SUM(cases) AS cases
    FROM covid_data
    GROUP BY region, month
''')

# Create a view to report deaths by region and month
conn.execute('''
    CREATE VIEW deaths_by_region_months AS
    SELECT region, strftime('%Y-%m', date) AS month,
           SUM(deaths) AS deaths
    FROM covid_data
    GROUP BY region, month
''')

# Query the cases and deaths by region and month
result = conn.execute('''
    SELECT c.region, c.month, c.cases, d.deaths
    FROM cases_by_region_months c
    JOIN deaths_by_region_months d ON c.region = d.region AND c.month = d.month
''')

# Print the results
for row in result:
    print(row)

('1', '-3339-04', 467, 0.0)
('1', '-3440-05', 375, 0.0)
('1', '-3518-01', 231, 0.0)
('1', '-3541-09', 258, 0.0)
('1', '-3548-01', 245, 0.0)
('1', '-3615-05', 233, 0.0)
('1', '-3639-08', 268, 0.0)
('1', '-3647-10', 167, 0.0)
('1', '-3705-11', 341, 0.0)
('1', '-3722-04', 246, 0.0)
('1', '-3745-10', 263, 0.0)
('1', '-3803-07', 246, 0.0)
('1', '-3803-12', 178, 0.0)
('1', '-3811-09', 191, 0.0)
('1', '-3814-07', 193, 0.0)
('1', '-3824-07', 148, 0.0)
('1', '-3877-11', 225, 0.0)
('1', '-3881-04', 142, 0.0)
('1', '-3887-09', 115, 0.0)
('1', '-3902-08', 90, 0.0)
('1', '-3951-05', 98, 0.0)
('1', '-3969-10', 351, 0.0)
('1', '-3997-10', 204, 0.0)
('1', '-4030-11', 127, 0.0)
('1', '-4088-06', 434, 0.0)
('1', '-4109-08', 294, 0.0)
('1', '-4112-09', 198, 0.0)
('1', '-4113-11', 110, 0.0)
('1', '-4124-09', 469, 0.0)
('1', '-4162-02', 380, 0.0)
('1', '-4163-11', 360, 0.0)
('1', '-4168-05', 385, 0.0)
('1', '-4168-06', 173, 0.0)
('1', '-4172-09', 184, 0.0)
('1', '-4175-02', 313, 0.0)
('1', '-4179-01', 176,

In [ ]:
# QUESTION 5: Vaccination rates for each vaccine






OperationalError: no such column: vaccine_type

In [ ]:
query = """
SELECT region, month, vaccine, AVG(Vaccination_Rate)
FROM vaccination
GROUP BY region, month, vaccine
"""

# execute the query
cursor.execute(query)

# fetch the results and print them
results = cursor.fetchall()
for row in results:
    region = row[0]
    month = row[1]
    vaccine = row[2]
    rate = row[3]
    print(f"{region}, {month}, {vaccine}: {rate}")

1000, 250, 2000: None
10000, 2500, 20000: None
10008, 2502, 20016: None
10016, 2504, 20032: None
10024, 2506, 20048: None
10032, 2508, 20064: None
10040, 2510, 20080: None
10048, 2512, 20096: None
10056, 2514, 20112: None
10064, 2516, 20128: None
10072, 2518, 20144: None
1008, 252, 2016: None
10080, 2520, 20160: None
10088, 2522, 20176: None
10096, 2524, 20192: None
10104, 2526, 20208: None
10112, 2528, 20224: None
10120, 2530, 20240: None
10128, 2532, 20256: None
10136, 2534, 20272: None
10144, 2536, 20288: None
10152, 2538, 20304: None
1016, 254, 2032: None
10160, 2540, 20320: None
10168, 2542, 20336: None
10176, 2544, 20352: None
10184, 2546, 20368: None
10192, 2548, 20384: None
10200, 2550, 20400: None
10208, 2552, 20416: None
10216, 2554, 20432: None
10224, 2556, 20448: None
10232, 2558, 20464: None
1024, 256, 2048: None
10240, 2560, 20480: None
10248, 2562, 20496: None
10256, 2564, 20512: None
10264, 2566, 20528: None
10272, 2568, 20544: None
10280, 2570, 20560: None
10288, 2572,

In [ ]:
# select the region, month, vaccine, and vaccination rate as a percentage
query = '''SELECT region, month, vaccine, CAST(Vaccination_Rate AS REAL) * 100
           FROM vaccination LIMIT 100'''

cursor.execute(query)
result = cursor.fetchall()

# print the results
for row in result:
    print(row)

('8', '2', '16', None)
('16', '4', '32', None)
('24', '6', '48', None)
('32', '8', '64', None)
('40', '10', '80', None)
('48', '12', '96', None)
('56', '14', '112', None)
('64', '16', '128', None)
('72', '18', '144', None)
('80', '20', '160', None)
('88', '22', '176', None)
('96', '24', '192', None)
('104', '26', '208', None)
('112', '28', '224', None)
('120', '30', '240', None)
('128', '32', '256', None)
('136', '34', '272', None)
('144', '36', '288', None)
('152', '38', '304', None)
('160', '40', '320', None)
('168', '42', '336', None)
('176', '44', '352', None)
('184', '46', '368', None)
('192', '48', '384', None)
('200', '50', '400', None)
('208', '52', '416', None)
('216', '54', '432', None)
('224', '56', '448', None)
('232', '58', '464', None)
('240', '60', '480', None)
('248', '62', '496', None)
('256', '64', '512', None)
('264', '66', '528', None)
('272', '68', '544', None)
('280', '70', '560', None)
('288', '72', '576', None)
('296', '74', '592', None)
('304', '76', '608', Non

In [ ]:
for row in rows:
    if row[2] is None:
        row[2] = 0
    if row[3] is None:
        row[3] = 0


NameError: name 'rows' is not defined

In [ ]:
# Execute the query to fetch data
cursor.execute("SELECT region, month, vaccine, AVG(Vaccination_Rate) AS avg_rate FROM vaccination GROUP BY region, month, vaccine")

# Fetch all rows and store them in the `rows` variable
rows = cursor.fetchall()

# Convert None values to 0
rows = [(region, month, vaccine, avg_rate if avg_rate is not None else 0) for region, month, vaccine, avg_rate in rows]

# Print the rows variable to verify the data
print(rows)


[('1000', '250', '2000', 0), ('10000', '2500', '20000', 0), ('10008', '2502', '20016', 0), ('10016', '2504', '20032', 0), ('10024', '2506', '20048', 0), ('10032', '2508', '20064', 0), ('10040', '2510', '20080', 0), ('10048', '2512', '20096', 0), ('10056', '2514', '20112', 0), ('10064', '2516', '20128', 0), ('10072', '2518', '20144', 0), ('1008', '252', '2016', 0), ('10080', '2520', '20160', 0), ('10088', '2522', '20176', 0), ('10096', '2524', '20192', 0), ('10104', '2526', '20208', 0), ('10112', '2528', '20224', 0), ('10120', '2530', '20240', 0), ('10128', '2532', '20256', 0), ('10136', '2534', '20272', 0), ('10144', '2536', '20288', 0), ('10152', '2538', '20304', 0), ('1016', '254', '2032', 0), ('10160', '2540', '20320', 0), ('10168', '2542', '20336', 0), ('10176', '2544', '20352', 0), ('10184', '2546', '20368', 0), ('10192', '2548', '20384', 0), ('10200', '2550', '20400', 0), ('10208', '2552', '20416', 0), ('10216', '2554', '20432', 0), ('10224', '2556', '20448', 0), ('10232', '2558'

In [ ]:
#QUESTION 5.2 Country and Quarter


# Query to report COVID-19 cases and deaths by country and quarter
query = """
SELECT strftime('%Y-Q', Date) as Quarter, Region, SUM(Cases) as Total_Cases, SUM(Deaths) as Total_Deaths
FROM covid_data
GROUP BY Quarter, Region
"""

# Execute the query
cursor.execute(query)

# Fetch the results
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

(None, '1', 41, 0.0)
(None, '10', 13, 0.0)
(None, '11', 1, 0.0)
(None, '12', 22, 0.0)
(None, '2', 17, 0.0)
(None, '3', 55, 0.0)
(None, '4', 927, 0.0)
(None, '5', 1426, 0.0)
(None, '6', 438, 0.0)
(None, '7', None, None)
(None, '8', 0, 0.0)
(None, '9', 0, 0.0)
('-3339-Q', '1', 467, 0.0)
('-3440-Q', '1', 375, 0.0)
('-3518-Q', '1', 231, 0.0)
('-3541-Q', '1', 258, 0.0)
('-3548-Q', '1', 245, 0.0)
('-3571-Q', '2', 381, 0.0)
('-3615-Q', '1', 233, 0.0)
('-3639-Q', '1', 268, 0.0)
('-3641-Q', '2', 8, 0.0)
('-3647-Q', '1', 167, 0.0)
('-3705-Q', '1', 341, 0.0)
('-3722-Q', '1', 246, 0.0)
('-3745-Q', '1', 263, 0.0)
('-3803-Q', '1', 424, 0.0)
('-3811-Q', '1', 191, 0.0)
('-3814-Q', '1', 193, 0.0)
('-3824-Q', '1', 148, 0.0)
('-3849-Q', '2', 276, 0.0)
('-3870-Q', '3', 303, 0.0)
('-3871-Q', '3', 358, 0.0)
('-3877-Q', '1', 225, 0.0)
('-3878-Q', '3', 336, 0.0)
('-3881-Q', '1', 142, 0.0)
('-3887-Q', '1', 115, 0.0)
('-3902-Q', '1', 90, 0.0)
('-3905-Q', '3', 355, 0.0)
('-3929-Q', '3', 290, 0.0)
('-3932-Q', '3'